In [5]:
import numpy as np
import matplotlib.pyplot as plt

In [15]:
def Curve(self,Ip,Td, Indice=0,Alpha=0,Beta=0,*name):
        # indice: indica el método de la curva
        # '1', Standard Inverse (IEC)           
        # '2', Very Inverse (IEC)               
        # '3', Extremely Inverse (IEC)          
        # '4', Long Time Standard (IEC)   
        # Ip, Pickup current
        # Td, TMS: time multiplier setting
        if Indice <= 11:
            method_name = 'curve_' + str(Indice)
            method = getattr(self, method_name, lambda: 'Invalid')
            [a, b] = method()
            self.__curveName = a
            self.__alpha = b[0]
            self.__beta = b[1]

        else:
            self.__curveName = name
            self.__beta = Beta
            self.__alpha = Alpha
            self.__ip = Ip  # Pickup current
            self.__td = Td  # Time
            
        # name, beta, alpha
        def curve_1(self):  # '1', Standard Inverse (IEC)
            return ['Standard Inverse (IEC)', [0.14, 0.02]]

        def curve_2(self):  # '2', Very Inverse (IEC)
            return ['Very Inverse (IEC)', [13.5, 1]]

        def curve_3(self):  # '3', Extremely Inverse (IEC)
            return ['Extremely Inverse (IEC)', [80, 2]]

        def curve_4(self):  # '4', Long Time Standard (IEC)
            return ['Long Time Standard (IEC)', [120, 1]]
        
        def mathCurve(self, I_measured):
            return  self.__td * (self.__beta / (np.power((I_measured / self.__ip), self.__alpha) - 1)) * 1000  # ms
        
        def graficar(self):
            I = np.linspace(1, 1000, 1000)
            T = self.mathCurve(I)
            print(np.size(T))
            # Add a legend
            plt.loglog(I, T)
            plt.grid(True, which="both", ls="-")
            plt.legend('TC curve')
            plt.xlabel('I')
            plt.ylabel('T')
            plt.title('Curva TC')
            # Show the plot
        

In [16]:
def States(self, T_spam, I_pick, Td, CurveDelay, Index=1, Alpha=0.02, Beta=0.14, *name):
    self._curve= Curve(I_pick, Td, Index, Alpha, Beta, name) 
    self.__timeEl = 0  # time elapsed in seconds
    self.__sampleTime = T_spam  # sample time in seconds
    self.__iPick = I_pick  # Pick up current in Amperes
    self.__delayCurve = CurveDelay  # seconds
    
    def __avanceTiempo(self):
        self.__timeEl = self.__timeEl + self.__sampleTime
    
    def __restartCounter(self):
        self.__timeEl = 0
    
    def __checkCurrent(self, I_measured):
        if abs(1000*I_measured) > self.__iPick:
            #Detectó una falla
            self.__addTime()
        else:
            #No detectó
            self.__restartCounter()
    
    def Status(self, I_measured):
        #####List of messages####
        # return 0, if 'Normal' state
        # return 1, if 'Stand By' state
        # return 2, if 'Lock out' state
        msg = 0
        self.__checkCurrent(I_measured)
        if self.__iPick < 1000 * abs(I_measured):
            # Detectó una falla, calculó el tiempo con la función Curve y le sumó el delay
            tCut = self.__curve.mathCurve(1000 * abs(I_measured)) + self.__delayCurve  # miliseconds
            if self.__timeEl >= tCut:
                # Si el tiempo actual es mayor al calculado--> Lock out
                msg = 2
            else:
                msg = 1
        return msg